In [1]:
import time
import networkx as nx
import numpy as np
import pandas as pd
import math
import os
import codecs
import seaborn as sns
import matplotlib as plt

In [2]:
'''filter out some self-login events consistent with the source and target devices'''
def sim_red(file,savefile):
    savefile.truncate(0)
    savefile.writelines('time,suser,sdomain, source,tuser,tdomain,target,auth_type,log_type,logon,authon\n')
    j=0
    oe = 0
    with open(file, 'r', encoding='utf-8') as fi:
        start = time.time()
        while True:
            line = fi.readline()
            oe+=1
            if not line:
                break
            l = line.strip().split(',')
            if(l[5]=="?" or l[6]=="?" or l[7]=="LogOff" ):
                continue
            if("U" in l[1] and "U" in l[2]):
                s_ud = l[1].strip().split('@')
                t_ud = l[2].strip().split('@')
                if(s_ud[0]=="ANONYMOUS LOGON" or t_ud[0]=="ANONYMOUS LOGON" or l[3]==l[4] or s_ud[0]!=t_ud[0]):
                    continue
                if(s_ud[1]=='?'or t_ud[1]=='?'):
                    continue
                logon = 0 if l[7]=="LogOn" else 1
                authon = 0 if l[8]=="Success" else 1
                savefile.writelines(str(l[0])+ ','+str(s_ud[0]) + ','+str(s_ud[1])+ \
                                         ','+str(l[3])+ ','+str(t_ud[0]) + ','+str(t_ud[1])+ \
                                         ','+str(l[4])+ ','+str(l[5])+','+ str(l[6])+ ','+ str(logon)+ \
                                         ','+str(authon)+ '\n')
            end = time.time()
            if(float(end-start)>j*100):
                print("print current read time : " + l[0])
                j+=1
        fi.close()
        savefile.close()
    return oe

In [3]:
PATH = 'G:/EngD/data/Dynamic Cybersecurity/data/auth.txt'# add the path of auth.txt
newauth = codecs.open('../data/LANL' + '/nedge.txt', 'w+', 'utf-8')
original_events = sim_red(PATH,newauth)

print current read time : 1
print current read time : 237762
print current read time : 508469
print current read time : 719462
print current read time : 951703
print current read time : 1172640
print current read time : 1371852
print current read time : 1655857
print current read time : 1884142
print current read time : 1923456
print current read time : 2044366
print current read time : 2306473
print current read time : 2507409
print current read time : 2742008
print current read time : 2942336
print current read time : 2942336
print current read time : 3118820
print current read time : 3347343
print current read time : 3458658
print current read time : 3517352
print current read time : 3685668
print current read time : 3822687
print current read time : 4002615
print current read time : 4204559
print current read time : 4239307
print current read time : 4239307
print current read time : 4239307
print current read time : 4323139
print current read time : 4523227
print current read time 

In [4]:
def iter_count(file_name):
    from itertools import (takewhile, repeat)
    buffer = 1024 * 1024
    with open(file_name) as f:
        buf_gen = takewhile(lambda x: x, (f.read(buffer) for _ in repeat(None)))
        return sum(buf.count('\n') for buf in buf_gen)

In [5]:
#original events
print("the number of all events:",original_events)

the number of all events: 1051430460


In [6]:
#filter events
number_nevents = iter_count('../data/LANL/nedge.txt')
print("the number of filter events:",number_nevents)

the number of filter events: 101662851


In [8]:
#filter some obviously normal events are also removed, such as frequent connections that occur greater than 24 times each day
fre_events = {}#greater than 24 times each day
Fre_edge = set()
half_events = {}#
filter_events = '../data/LANL/TCG_LANL.txt'
with open('../data/LANL/nedge.txt', 'r', encoding='utf-8') as fi:
    line = fi.readline()
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        str_line = str(l[1])+','+str(l[2])+','+str(l[3])+','+str(l[4])+','+str(l[5])+','+str(l[6])+','+str(l[7])+','+str(l[8])+','+str(l[9])+','+str(l[10])
        if str_line in fre_events:
            if(int(l[0])-fre_events[str_line][0]<=86400):
                fre_events[str_line][1] += 1
            elif(fre_events[str_line][1]>=24):
                Fre_edge.add(str_line)
                fre_events[str_line] = [int(l[0]),0]
                half_events[str_line] += 1
        else:
            fre_events[str_line] = [int(l[0]),1]
            half_events[str_line] = 1
    fi.close()

In [9]:
hf_events = set()#one event occur greater than half of all time
for e in half_events:
    if(half_events[e]>=5011198/172800):
        hf_events.add(e)

In [10]:
with open('../data/LANL/nedge.txt', 'r', encoding='utf-8') as fi:
    line = fi.readline()
    ofr = open(filter_events,'w+')
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        str_line = str(l[1])+','+str(l[2])+','+str(l[3])+','+str(l[4])+','+str(l[5])+','+str(l[6])+','+str(l[7])+','+str(l[8])+','+str(l[9])+','+str(l[10])
        if(str_line in Fre_edge):
            continue
        else:
            ofr.write(line)
    fi.close()
    ofr.close()

In [22]:
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team,'r').readlines())
num_redteam = 0
with open(filter_events, 'r', encoding='utf-8') as fi:
    line = fi.readline()
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        str_line =str(l[0])+','+str(l[1])+','+str(l[2])+','+str(l[3])+','+str(l[6])+'\n'
        if str_line in red_file:
            num_redteam+=1
    fi.close()

In [23]:
print('number of red_team',num_redteam)

number of red_team 702


In [24]:
num_filter = iter_count(filter_events)
print("filt frequency and half date events",num_filter)

filt frequency and half date events 36946085


# statistical

In [1]:
#statistical_TCG
filter_events = '../data/LANL/TCG_LANL.txt'

In [2]:
num_users= set()
num_devices = set()
with open(filter_events, 'r', encoding='utf-8') as fi:
    line = fi.readline()
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        num_users.add(l[1])
        num_users.add(l[4])
        num_devices.add(l[3])
        num_devices.add(l[6])
    fi.close()

In [3]:
print("numbers of users in preprocess:",len(num_users))
print("numbers of devices in preprocess:",len(num_devices))

numbers of users in preprocess: 10840
numbers of devices in preprocess: 15343


In [4]:
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team,'r').readlines())

In [6]:
j=0
num_redteam = 0
origin_file_redteam = '../data/LANL/redteam.txt'
logon_list = []
red_users = set()
red_devices = set()
flag=1
flag_red = 1
with open(filter_events, 'r', encoding='utf-8') as fi:
    line = fi.readline()
    ofr = open(origin_file_redteam,'w+')
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        '''if(int(l[0])<150885):
            continue'''
        str_line =str(l[0])+','+str(l[1])+','+str(l[2])+','+str(l[3])+','+str(l[6])+'\n'
        #print(str_line)
        if(flag):
            start_t = l[0]
            flag = 0
        last_t=l[0]
        if str_line in red_file:
            #label[j]=1
            if(flag_red):
                start_redt = l[0]
                flag_red = 0
            last_redt = l[0]
            num_redteam+=1
            red_users.add(l[1])
            red_users.add(l[4])
            red_devices.add(l[3])
            red_devices.add(l[6])
            logon_list.append(l[-2])
            ofr.write(line)
        j+=1
    fi.close()
    ofr.close()

In [7]:
print("The number of lateral movement:",num_redteam)
print("The number of red users:",len(red_users))
print("The number of red devices:",len(red_devices))
print("The time span of all log:",(int(last_t)-int(start_t))/3600/24)
print("The time span of red team:",(int(last_redt)-int(start_redt))/3600/24)

The number of lateral movement: 702
The number of red users: 98
The number of red devices: 299
The time span of all log: 57.99997685185185
The time span of red team: 27.849097222222223


In [ ]:
#statistical_AIG
num_users= set()
num_devices = set()
AIGfile = "opt_TUDCC3.txt"
with open(AIGfile, 'r', encoding='utf-8') as fi:
    line = fi.readline()
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        num_users.add(l[1])
        num_users.add(l[4])
        num_devices.add(l[3])
        num_devices.add(l[6])
    fi.close()

In [ ]:
j=0
#label = np.zeros(1051430459)
num_redteam = 0
origin_file_redteam = '../data/LANL/origin_file_redteam.txt'
red_team = '../data/LANL/nredteam.txt'
red_file = set(open(red_team,'r').readlines())
logon_list = []
red_users = set()
red_devices = set()
flag=1
flag_red = 1
with open(AIGfile, 'r', encoding='utf-8') as fi:
    line = fi.readline()
    ofr = open(origin_file_redteam,'w+')
    while True:
        line = fi.readline()
        if not line:
            break
        l = line.strip().split(',')
        str_line =str(l[0])+','+str(l[1])+','+str(l[2])+','+str(l[3])+','+str(l[6])+'\n'
        if(flag):
            start_t = l[0]
            flag = 0
        last_t=l[0]
        if str_line in red_file:
            #label[j]=1
            if(flag_red):
                start_redt = l[0]
                flag_red = 0
            last_redt = l[0]
            num_redteam+=1
            red_users.add(l[1])
            red_users.add(l[4])
            red_devices.add(l[3])
            red_devices.add(l[6])
            logon_list.append(l[-2])
            ofr.write(line)
        j+=1
    fi.close()
    ofr.close()

In [ ]:
print("numbers of users in preprocess:",len(num_users))
print("numbers of devices in preprocess:",len(num_devices))
print("The number of lateral movement:",num_redteam)
print("The number of red users:",len(red_users))
print("The number of red devices:",len(red_devices))
print("The time span of all log:",(int(last_t)-int(start_t))/3600/24)
print("The time span of red team:",(int(last_redt)-int(start_redt))/3600/24)

In [ ]:
iter_count(red_team)

In [ ]:
TUDCCPATH = './opt_TUDCC.txt'
iter_count(TUDCCPATH)